In [53]:
# The EC equation is: y^2 = x^3 + ax + b
## A finite field and the order of the group must be announced publicly.
## Order of the group: Number of points on the curve
## Finite field: How much do you modulo a number by.

# Generator Point G: {x = 55066263022277343669578718895168534326250603453777594175500187360389116729240, y = 32670510020758816978083085130507043184471273380659243275938904335757337482424}}
# Order n: 115792089237316195423570985008687907852837564279074904382605163141518161494337

# n = 115792089237316195423570985008687907852837564279074904382605163141518161494337 (Scalar Field)
# p = 115792089237316195423570985008687907853269984665640564039457584007908834671663 (Base Field)

# '''
# Now what are these n and p numbers?

# Well, n is the scalar field and p is the base field.

# n = https://github.com/matter-labs/era-zkevm_circuits/blob/v1.4.1/src/ecrecover/secp256k1/fr.rs
# p = https://github.com/matter-labs/era-zkevm_circuits/blob/v1.4.1/src/ecrecover/secp256k1/fq.rs
 
# Base field of an elliptic curve is the field over which the curve is defined. 
# The base field size thereby defines the number of elements of the finite field.
# Scalar field is the field of scalars used in the operations performed on the curve, such as point addition, scalar multiplication and pairings
# '''

# The private key is a random integer between [0..n-1]
# And then, the public key would be a point on the Elliptic curve, privKey * G

# For the `secp256k1` curve, the private key is 256-bit integer (32 bytes) and the compressed public key is 257-bit integer (~33 bytes)

In [54]:
# pip install pycoin

import hashlib
import pycoin, secrets

import pycoin.ecdsa
import pycoin.ecdsa.secp256k1

secp256k1_generator = pycoin.ecdsa.secp256k1.secp256k1_generator

def sha3_256Hash(msg):
    hashBytes = hashlib.sha3_256(msg.encode("utf8")).digest()
    return int.from_bytes(hashBytes, byteorder="big")

def signECDSAsecp256k1(msg, privKey):
    msgHash = sha3_256Hash(msg)
    signature = pycoin.ecdsa.secp256k1.Generator.sign(secp256k1_generator, privKey, msgHash)
    return signature

def verifyECDSAsecp256k1(msg, signature, pubKey):
    msgHash = sha3_256Hash(msg)
    valid = pycoin.ecdsa.secp256k1.Generator.verify(secp256k1_generator, pubKey, msgHash, signature)
    return valid

print(sha3_256Hash("Hello Boojum"))

# Generate a signature for a message using ECDSA
msg = "Hello Boojum"
privKey = secrets.randbelow(secp256k1_generator.order())
signature = signECDSAsecp256k1(msg, privKey)

print("Message: ", msg)
print("Private Key: ", privKey)
print("Signature: r= " + hex(signature[0]) + " , s= " + hex(signature[1]))

# ECDSA verify signature
pubKey = secp256k1_generator.raw_mul(privKey)
valid = verifyECDSAsecp256k1(msg, signature, pubKey)

print("\nMessage: ", msg)
print("Public Key: (" + hex(pubKey[0]) + ", " + hex(pubKey[1]) + ")")
print("Signature Valid?", valid)

# Negative test
msg = "Tampered Message"
valid = verifyECDSAsecp256k1(msg, signature, pubKey)
print("\n message: ", msg)
print("Signature: (tampered msg) is it valid?? ", valid)

15795942572089539261785026942139819340104458755837367492447218388857379368638
Message:  Hello Boojum
Private Key:  14557687400911315942777225618197320611973559191654161877905159191096137924645
Signature: r= 0x5a1f45320c2ff190b74aea65ffcc434953162e6603500f37b14236e02302b2c0 , s= 0xf4f4fa998eca45a0a1d1211b0c6571cb63569cb954a8f534243d57afe63b7918

Message:  Hello Boojum
Public Key: (0x214a5eea66b73b910de29101fd12f8c6b8c22a08af58b31b62a0fddc13449e9d, 0xfae89e697ce6894034a74ffcf62c08a3dfebceba51b80a2b40cb804a0534d717)
Signature Valid? True

 message:  Tampered Message
Signature: (tampered msg) is it valid??  False
